In [6]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *


import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
experiment = Experiment.objects.last()

In [8]:
master_df = get_all_frames(experiment)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [02:31<00:00,  7.95s/it]


In [ ]:
!ls /shared/outputs

In [5]:
plot_all_models(master_df, notebook=False)

,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
4,0.000001,feb22/mlp,128.863531,0,0,8.984528,119.879002,119.879002,119.879002,216.151513,2,large_game_2-feb22mlp-100
10,1.000000,feb22/mlp,88.318605,0,0,79.539632,8.778973,8.778973,8.778973,12.737449,2,large_game_2-feb22mlp-100
13,2.000000,feb22/mlp,88.109647,0,0,83.532859,4.576788,4.576788,4.576788,16.121980,2,large_game_2-feb22mlp-100
19,3.000000,feb22/mlp,89.781968,0,0,79.032544,10.749424,10.749424,10.749424,29.471459,2,large_game_2-feb22mlp-100
25,4.000000,feb22/mlp,89.242305,0,0,72.549998,16.692307,16.692307,16.692307,41.660628,2,large_game_2-feb22mlp-100
31,5.000000,feb22/mlp,91.342317,0,0,75.230457,16.111860,16.111860,16.111860,42.211642,2,large_game_2-feb22mlp-100
37,6.000000,feb22/mlp,91.712889,0,0,87.389538,4.323351,4.323351,4.323351,23.781535,2,large_game_2-feb22mlp-100


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
1,0.000001,feb22/mlp,163.997600,1,1,67.725089,96.272511,96.272511,96.272511,216.151513,2,large_game_2-feb22mlp-100
7,1.000000,feb22/mlp,151.104513,1,1,147.146037,3.958476,3.958476,3.958476,12.737449,2,large_game_2-feb22mlp-100
16,2.000000,feb22/mlp,162.460147,1,1,150.914955,11.545192,11.545192,11.545192,16.121980,2,large_game_2-feb22mlp-100
22,3.000000,feb22/mlp,164.676763,1,1,145.954728,18.722035,18.722035,18.722035,29.471459,2,large_game_2-feb22mlp-100
28,4.000000,feb22/mlp,163.472268,1,1,138.503946,24.968321,24.968321,24.968321,41.660628,2,large_game_2-feb22mlp-100
34,5.000000,feb22/mlp,166.129494,1,1,140.029712,26.099782,26.099782,26.099782,42.211642,2,large_game_2-feb22mlp-100
40,6.000000,feb22/mlp,166.204469,1,1,146.746286,19.458183,19.458183,19.458183,23.781535,2,large_game_2-feb22mlp-100


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
1,0.000001,feb22/mlp,92.329579,0,0,-3.231409,95.560988,95.560988,95.560988,162.172452,2,large_game_2-feb22mlp-101
7,1.000000,feb22/mlp,120.673464,0,0,108.241388,12.432075,12.432075,12.432075,18.301297,2,large_game_2-feb22mlp-101
16,2.000000,feb22/mlp,123.198097,0,0,113.306733,9.891364,9.891364,9.891364,15.539311,2,large_game_2-feb22mlp-101
19,3.000000,feb22/mlp,125.861509,0,0,116.648137,9.213372,9.213372,9.213372,10.197758,2,large_game_2-feb22mlp-101
25,4.000000,feb22/mlp,125.436326,0,0,119.299547,6.136778,6.136778,6.136778,6.136778,2,large_game_2-feb22mlp-101
34,5.000000,feb22/mlp,132.994099,0,0,119.773668,13.220431,13.220431,13.220431,16.070512,2,large_game_2-feb22mlp-101
40,6.000000,feb22/mlp,127.464383,0,0,116.921342,10.543041,10.543041,10.543041,12.380241,2,large_game_2-feb22mlp-101
43,7.000000,feb22/mlp,125.888237,0,0,114.505266,11.382971,11.382971,11.382971,19.133429,2,large_game_2-feb22mlp-101
52,8.000000,feb22/mlp,125.968768,0,0,111.276150,14.692618,14.692618,14.692618,21.872843,2,large_game_2-feb22mlp-101


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
3,0.000001,feb22/mlp,200.007278,1,1,133.395814,66.611464,66.611464,66.611464,162.172452,2,large_game_2-feb22mlp-101
10,1.000000,feb22/mlp,142.851125,1,1,136.981904,5.869222,5.869222,5.869222,18.301297,2,large_game_2-feb22mlp-101
13,2.000000,feb22/mlp,149.890278,1,1,144.242332,5.647947,5.647947,5.647947,15.539311,2,large_game_2-feb22mlp-101
22,3.000000,feb22/mlp,150.897622,1,1,149.913235,0.984387,0.984387,0.984387,10.197758,2,large_game_2-feb22mlp-101
28,4.000000,feb22/mlp,151.648839,1,1,151.828569,-0.179731,0.000000,0.000000,6.136778,2,large_game_2-feb22mlp-101
31,5.000000,feb22/mlp,153.551917,1,1,150.701836,2.850081,2.850081,2.850081,16.070512,2,large_game_2-feb22mlp-101
37,6.000000,feb22/mlp,147.951534,1,1,146.114334,1.837200,1.837200,1.837200,12.380241,2,large_game_2-feb22mlp-101
46,7.000000,feb22/mlp,148.902280,1,1,141.151822,7.750459,7.750459,7.750459,19.133429,2,large_game_2-feb22mlp-101
49,8.000000,feb22/mlp,145.357881,1,1,138.177656,7.180225,7.180225,7.180225,21.872843,2,large_game_2-feb22mlp-101


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
1,0.000001,feb22/mlp,157.823274,0,0,108.718293,49.104981,49.104981,49.104981,158.912587,2,large_game_2-feb22mlp-102
7,1.000000,feb22/mlp,123.355672,0,0,116.367212,6.988460,6.988460,6.988460,18.838137,2,large_game_2-feb22mlp-102
13,2.000000,feb22/mlp,127.809891,0,0,123.392482,4.417409,4.417409,4.417409,9.418891,2,large_game_2-feb22mlp-102
19,3.000000,feb22/mlp,129.886213,0,0,126.386628,3.499585,3.499585,3.499585,3.499585,2,large_game_2-feb22mlp-102
25,4.000000,feb22/mlp,130.257721,0,0,128.823059,1.434662,1.434662,1.434662,1.434662,2,large_game_2-feb22mlp-102
34,5.000000,feb22/mlp,130.013037,0,0,127.952891,2.060146,2.060146,2.060146,8.620500,2,large_game_2-feb22mlp-102
37,6.000000,feb22/mlp,128.981021,0,0,127.703870,1.277151,1.277151,1.277151,1.948598,2,large_game_2-feb22mlp-102
43,7.000000,feb22/mlp,129.399430,0,0,127.208668,2.190761,2.190761,2.190761,4.826429,2,large_game_2-feb22mlp-102
49,8.000000,feb22/mlp,130.009319,0,0,126.731843,3.277476,3.277476,3.277476,7.178185,2,large_game_2-feb22mlp-102


...

,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
4,0.000001,feb22/mlp,133.800800,1,1,62.715369,71.085431,71.085431,71.085431,257.576831,2,large_game_2_hide-feb22mlp-107
10,1.000000,feb22/mlp,131.987159,1,1,124.046025,7.941135,7.941135,7.941135,7.941135,2,large_game_2_hide-feb22mlp-107
16,2.000000,feb22/mlp,130.577196,1,1,126.823730,3.753466,3.753466,3.753466,11.424017,2,large_game_2_hide-feb22mlp-107
22,3.000000,feb22/mlp,134.158441,1,1,128.622026,5.536415,5.536415,5.536415,9.990964,2,large_game_2_hide-feb22mlp-107
28,4.000000,feb22/mlp,134.425338,1,1,129.312341,5.112997,5.112997,5.112997,10.203444,2,large_game_2_hide-feb22mlp-107
34,5.000000,feb22/mlp,134.672086,1,1,129.771660,4.900426,4.900426,4.900426,9.641546,2,large_game_2_hide-feb22mlp-107
40,6.000000,feb22/mlp,135.466056,1,1,130.353469,5.112587,5.112587,5.112587,9.096755,2,large_game_2_hide-feb22mlp-107
46,7.000000,feb22/mlp,132.027600,1,1,130.139398,1.888202,1.888202,1.888202,5.831112,2,large_game_2_hide-feb22mlp-107
52,8.000000,feb22/mlp,134.457235,1,1,130.420468,4.036767,4.036767,4.036767,8.380749,2,large_game_2_hide-feb22mlp-107


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
0,0.000001,feb22/mlp,153.729675,0,0,42.851130,110.878545,110.878545,110.878545,236.379849,2,large_game_2_hide-feb22mlp-108
7,1.000000,feb22/mlp,111.145995,0,0,100.144720,11.001274,11.001274,11.001274,15.882767,2,large_game_2_hide-feb22mlp-108
13,2.000000,feb22/mlp,108.662779,0,0,101.744596,6.918184,6.918184,6.918184,13.192910,2,large_game_2_hide-feb22mlp-108
19,3.000000,feb22/mlp,106.824546,0,0,99.837986,6.986560,6.986560,6.986560,15.305993,2,large_game_2_hide-feb22mlp-108
25,4.000000,feb22/mlp,106.854369,0,0,98.667595,8.186774,8.186774,8.186774,15.951651,2,large_game_2_hide-feb22mlp-108
34,5.000000,feb22/mlp,106.118124,0,0,97.701398,8.416726,8.416726,8.416726,17.431260,2,large_game_2_hide-feb22mlp-108
37,6.000000,feb22/mlp,101.525908,0,0,100.168941,1.356967,1.356967,1.356967,10.164613,2,large_game_2_hide-feb22mlp-108
43,7.000000,feb22/mlp,100.744168,0,0,100.129506,0.614662,0.614662,0.614662,9.095412,2,large_game_2_hide-feb22mlp-108


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
3,0.000001,feb22/mlp,195.959836,1,1,70.458533,125.501303,125.501303,125.501303,236.379849,2,large_game_2_hide-feb22mlp-108
10,1.000000,feb22/mlp,142.664760,1,1,137.783267,4.881493,4.881493,4.881493,15.882767,2,large_game_2_hide-feb22mlp-108
16,2.000000,feb22/mlp,150.395954,1,1,144.121228,6.274726,6.274726,6.274726,13.192910,2,large_game_2_hide-feb22mlp-108
22,3.000000,feb22/mlp,154.548039,1,1,146.228606,8.319433,8.319433,8.319433,15.305993,2,large_game_2_hide-feb22mlp-108
28,4.000000,feb22/mlp,154.607286,1,1,146.842408,7.764878,7.764878,7.764878,15.951651,2,large_game_2_hide-feb22mlp-108
31,5.000000,feb22/mlp,157.834960,1,1,148.820427,9.014534,9.014534,9.014534,17.431260,2,large_game_2_hide-feb22mlp-108
40,6.000000,feb22/mlp,157.311342,1,1,148.503696,8.807646,8.807646,8.807646,10.164613,2,large_game_2_hide-feb22mlp-108
46,7.000000,feb22/mlp,155.549503,1,1,147.068753,8.480750,8.480750,8.480750,9.095412,2,large_game_2_hide-feb22mlp-108


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
4,0.000001,feb22/mlp,223.430034,0,0,191.760168,31.669866,31.669866,31.669866,91.584643,2,large_game_2_hide-feb22mlp-109
7,1.000000,feb22/mlp,117.498470,0,0,111.618625,5.879845,5.879845,5.879845,10.593983,2,large_game_2_hide-feb22mlp-109
16,2.000000,feb22/mlp,118.561598,0,0,114.300370,4.261228,4.261228,4.261228,10.640105,2,large_game_2_hide-feb22mlp-109
19,3.000000,feb22/mlp,118.441240,0,0,116.334207,2.107033,2.107033,2.107033,4.433309,2,large_game_2_hide-feb22mlp-109
25,4.000000,feb22/mlp,119.509196,0,0,116.282549,3.226647,3.226647,3.226647,5.737838,2,large_game_2_hide-feb22mlp-109
34,5.000000,feb22/mlp,119.805635,0,0,117.097042,2.708592,2.708592,2.708592,5.566861,2,large_game_2_hide-feb22mlp-109
37,6.000000,feb22/mlp,117.628222,0,0,117.282462,0.345761,0.345761,0.345761,3.315105,2,large_game_2_hide-feb22mlp-109
43,7.000000,feb22/mlp,118.745043,0,0,117.602416,1.142627,1.142627,1.142627,3.588373,2,large_game_2_hide-feb22mlp-109
49,8.000000,feb22/mlp,118.732280,0,0,118.336482,0.395799,0.395799,0.395799,2.295358,2,large_game_2_hide-feb22mlp-109


,t,name,reward,br_player,player,Baseline,Regret,PositiveRegret,MaxPositiveRegret,ApproxNashConv,num_players,model
1,0.000001,feb22/mlp,88.895702,1,1,28.980925,59.914777,59.914777,59.914777,91.584643,2,large_game_2_hide-feb22mlp-109
10,1.000000,feb22/mlp,146.992079,1,1,142.277940,4.714138,4.714138,4.714138,10.593983,2,large_game_2_hide-feb22mlp-109
13,2.000000,feb22/mlp,150.267922,1,1,143.889045,6.378877,6.378877,6.378877,10.640105,2,large_game_2_hide-feb22mlp-109
22,3.000000,feb22/mlp,148.120110,1,1,145.793834,2.326276,2.326276,2.326276,4.433309,2,large_game_2_hide-feb22mlp-109
28,4.000000,feb22/mlp,147.063506,1,1,144.552315,2.511191,2.511191,2.511191,5.737838,2,large_game_2_hide-feb22mlp-109
31,5.000000,feb22/mlp,147.929483,1,1,145.071215,2.858269,2.858269,2.858269,5.566861,2,large_game_2_hide-feb22mlp-109
40,6.000000,feb22/mlp,147.794699,1,1,144.825355,2.969344,2.969344,2.969344,3.315105,2,large_game_2_hide-feb22mlp-109
46,7.000000,feb22/mlp,146.796768,1,1,144.351022,2.445746,2.445746,2.445746,3.588373,2,large_game_2_hide-feb22mlp-109
52,8.000000,feb22/mlp,146.880754,1,1,144.981195,1.899559,1.899559,1.899559,2.295358,2,large_game_2_hide-feb22mlp-109


In [ ]:
# s = ['_mlp', '_mlp_1', '_mlp_2', '_lstm', '_lstm_1', '_lstm_2', 'Straightforward']
fig = compare_best_responses(master_df)
special_save_fig(fig, "configs.png", fmt=None, dpi=300, tight=True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15, 5))
plt.imshow(list(q_network.output_layer.parameters())[0].data)
plt.colorbar()

In [ ]:
with open('/shared/outputs/feb7_large/mlp/evaluations/rewards_checkpoint_1500000_checkpoint_1500000_br_0_lstm_test.pkl', 'rb') as f:
    rewards = pickle.load(f)


In [ ]:
np.array(rewards['rewards'][0]).mean()

In [ ]:
master_df[['config', 'PositiveRegret']]

In [ ]:
x = master_df['config'].unique().tolist()
x.remove(None)
x.remove("Straightforward")
x

In [ ]:
for config in x:
    portfolio = [config, 'Straightforward']
    p = master_df.query('config in @portfolio and player == best_responder').copy()
    # NOT QUITE RIGHT
    p['frac'] = p['PositiveRegret'] / p['MaxPositiveRegret']
    p = p.groupby(['model', 't', 'player']).apply(lambda x: x['frac'].max())
    print(config, p.mean())

In [ ]:
y = list(x)
y.remove('_mlp_wide')
for config in y:
    portfolio = [config, '_mlp_wide', 'Straightforward']
    p = master_df.query('config in @portfolio and player == best_responder').copy()
    # NOT QUITE RIGHT
    p['frac'] = p['PositiveRegret'] / p['MaxPositiveRegret']
    p = p.groupby(['model', 't', 'player']).apply(lambda x: x['frac'].max())
    print(config, p.mean())

In [ ]:
z = list(y)
z.remove('_mlp_deep_wide')
for config in z:
    portfolio = [config, '_mlp_deep_wide', '_mlp_wide', 'Straightforward']
    p = master_df.query('config in @portfolio and player == best_responder').copy()
    # NOT QUITE RIGHT
    p['frac'] = p['PositiveRegret'] / p['MaxPositiveRegret']
    p = p.groupby(['model', 't', 'player']).apply(lambda x: x['frac'].max())
    print(config, p.mean())